In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime
import html5lib
import lxml

from utils import mp_to_minutes

In [2]:
import duckdb

DB_PATH = "assets/rookies.duckdb"
con = duckdb.connect(DB_PATH)

In [3]:
# Create tables for database if they do not exist
con.execute("""
CREATE TABLE IF NOT EXISTS draft_classes (
    draft_year INTEGER,
    player TEXT,
    player_id TEXT,
    rookie_season INTEGER
);

CREATE TABLE IF NOT EXISTS rookie_games (
    player_id TEXT,
    rookie_season INTEGER,
    game_no INTEGER,
    game_date DATE,
    MP DOUBLE,
    PTS INTEGER,
    TRB INTEGER,
    AST INTEGER,
    STL INTEGER,
    BLK INTEGER,
    TOV INTEGER,
    FG INTEGER,
    FGA INTEGER,
    TP INTEGER,
    TPA INTEGER,
    FT INTEGER,
    FTA INTEGER
);

CREATE TABLE IF NOT EXISTS player_bios (
    player_id TEXT,
    height_in INTEGER,
    weight_lb INTEGER,
    position TEXT
);

CREATE TABLE IF NOT EXISTS scrape_log (
    player_id TEXT,
    rookie_season INTEGER,
    status TEXT,
    error TEXT,
    last_attempt TIMESTAMP
);
""")


In [4]:
# one-time import of CSV data into database
con.execute("DELETE FROM draft_classes")
con.execute("INSERT INTO draft_classes SELECT * FROM read_csv_auto('assets/draft_classes.csv')")

if os.path.exists("assets/player_bios.csv"):
    con.execute("DELETE FROM player_bios")
    con.execute("INSERT INTO player_bios SELECT * FROM read_csv_auto('assets/player_bios.csv')")


In [5]:
# extract draft class from the url
def load_draft_class(draft_year):
    url = f"https://www.basketball-reference.com/draft/NBA_{draft_year}.html"

    df = pd.read_html(url)[0]
    df_links = pd.read_html(url, extract_links="body")[0]

    df.columns = [f"{a}_{b}".strip("_") for a, b in df.columns]
    df_links.columns = df.columns

    df = df[df["Round 1_Player"].notna()].copy()

    df["player"] = df["Round 1_Player"]
    df["player_id"] = df_links["Round 1_Player"].apply(
        lambda x: x[1].split("/")[-1].replace(".html", "")
        if isinstance(x, tuple) and x[1]
        else None
    )

    # 🔑 drop header rows like "Round 2"
    df = df[df["player_id"].notna()].copy()

    df["draft_year"] = draft_year
    df["rookie_season"] = draft_year + 1

    return df[["draft_year", "player", "player_id", "rookie_season"]]



In [6]:
# load or scrape draft class with caching
# def load_or_scrape_draft(year):
#     path = f"assets/drafts/draft_{year}.csv"
#     if os.path.exists(path):
#         return pd.read_csv(path)
    
#     df = load_draft_class(year)
#     df.to_csv(path, index=False)
#     return df

# function to scrape a single draft class
def scrape_and_save_year(year):
    df = load_draft_class(year)
    df.to_csv(f"assets/drafts/draft_{year}.csv", index=False)
    print(f"✓ saved {year}")


In [7]:
draft_classes = con.execute("SELECT * FROM draft_classes").df()

In [8]:
# override rookie seasons for specific players
rookie_overrides = {
    "griffbl01": 2011,  # drafted 2009, rookie season 2010–11
    "embiijo01": 2016,  # drafted 2014, rookie season 2015-16
}

In [9]:
def load_rookie_gamelog(player_id, season):
    first_letter = player_id[0]
    url = f"https://www.basketball-reference.com/players/{first_letter}/{player_id}/gamelog/{season}"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()

    dfs = pd.read_html(resp.text, attrs={"id": "player_game_log_reg"})
    if not dfs:
        raise ValueError("No tables found")

    df = dfs[0]

    df = df[df["Opp"].notna()].copy()
    df["MP"] = df["MP"].apply(mp_to_minutes)
    df = df[df["MP"].notna()].copy()

    return df


In [10]:
def insert_gamelog_into_db(player_id, rookie_season):
    gamelog = load_rookie_gamelog(player_id, rookie_season).copy()

    gamelog["player_id"] = player_id
    gamelog["rookie_season"] = rookie_season
    gamelog["game_no"] = np.arange(1, len(gamelog) + 1)

    # Normalize column names
    gamelog = gamelog.rename(columns={
        "Date": "game_date",
        "3P": "TP",
        "3PA": "TPA"
    })

    cols = ["player_id","rookie_season","game_no","game_date","MP","PTS","TRB","AST","STL","BLK","TOV","FG","FGA","TP","TPA","FT","FTA"]
    gamelog = gamelog[cols]

    con.execute("INSERT INTO rookie_games SELECT * FROM gamelog")


In [11]:
# function to scrape player bio data
# store height (inches), weight (lbs), position
def scrape_player_bio(player_id):
    first_letter = player_id[0]
    url = f"https://www.basketball-reference.com/players/{first_letter}/{player_id}.html"

    try:
        html = requests.get(url, timeout=10).text
        soup = BeautifulSoup(html, "lxml")

        pos = None
        height_in = None
        weight_lb = None

        # Position
        pos_tag = soup.find("strong", string="Position")
        if pos_tag:
            pos = pos_tag.next_sibling.strip().split("▪")[0].strip()

        # Height / Weight
        h_tag = soup.find("strong", string="Height")
        w_tag = soup.find("strong", string="Weight")

        if h_tag:
            h = h_tag.next_sibling.strip()
            feet, inches = h.split("-")
            height_in = int(feet) * 12 + int(inches)

        if w_tag:
            w = w_tag.next_sibling.strip().replace("lb", "").strip()
            weight_lb = int(w)

        return {
            "player_id": player_id,
            "height_in": height_in,
            "weight_lb": weight_lb,
            "position": pos,
        }

    except Exception as e:
        return {
            "player_id": player_id,
            "height_in": None,
            "weight_lb": None,
            "position": None,
        }


In [12]:
# count unique players in draft classes
unique_players = draft_classes["player_id"].dropna().unique()
len(unique_players)

1542

In [13]:
# load existing bios
# avoids redundant scraping
bios_path = "assets/player_bios.csv"

if os.path.exists(bios_path):
    player_bios = pd.read_csv(bios_path)
    done_ids = set(player_bios.player_id)
else:
    player_bios = pd.DataFrame(columns=["player_id", "height_in", "weight_lb", "position"])
    done_ids = set()


In [14]:
# identify players still needing bios
todo_bios = [pid for pid in unique_players if pid not in done_ids]
len(todo_bios)

1542

In [15]:
# scrape gamelogs into database
from datetime import datetime

def log_scrape(player_id, rookie_season, status, error=None):
    con.execute("""
        INSERT INTO scrape_log
        VALUES (?, ?, ?, ?, ?)
    """, [player_id, rookie_season, status, error, datetime.utcnow()])


# Find which (player_id, rookie_season) still need scraping
already_scraped = set(
    con.execute("SELECT DISTINCT player_id, rookie_season FROM rookie_games").fetchall()
)

draft_rows = con.execute("SELECT player_id, rookie_season FROM draft_classes").fetchall()

todo = [(pid, season) for pid, season in draft_rows if (pid, season) not in already_scraped]

print(f"Need to scrape {len(todo)} rookie seasons")


CHECKPOINT_EVERY = 25

for i, (player_id, rookie_season) in enumerate(todo, 1):
    try:
        insert_gamelog_into_db(player_id, rookie_season)
        log_scrape(player_id, rookie_season, "ok")

        if i % CHECKPOINT_EVERY == 0:
            print(f"Scraped {i}/{len(todo)}")

        time.sleep(random.uniform(10, 18))

    except requests.HTTPError as e:
        if "429" in str(e):
            print("Hit 429 — sleeping 10 minutes")
            time.sleep(600)
            continue
        else:
            log_scrape(player_id, rookie_season, "error", str(e))

    except Exception as e:
        log_scrape(player_id, rookie_season, "error", str(e))


Need to scrape 1542 rookie seasons


C:\Users\mehan\AppData\Local\Temp\ipykernel_30452\3495353397.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  """, [player_id, rookie_season, status, error, datetime.utcnow()])


In [16]:
con = duckdb.connect("assets/rookies.duckdb")

con.execute("SHOW TABLES").fetchall()

[('draft_classes',), ('player_bios',), ('rookie_games',), ('scrape_log',)]

In [17]:
for table in ["draft_classes", "rookie_games", "player_bios", "scrape_log"]:
    count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {count}")

draft_classes: 1542
rookie_games: 0
player_bios: 0
scrape_log: 1542


In [18]:
# inspect scrape status
con.execute("""
SELECT status, COUNT(*) 
FROM scrape_log
GROUP BY status
""").fetchall()

[('error', 1542)]

In [19]:
con.execute("""
SELECT error, COUNT(*) 
FROM scrape_log
GROUP BY error
ORDER BY COUNT(*) DESC
LIMIT 10
""").df()

,error,count_star()
0,403 Client Error: Forbidden for url: https://w...,1
1,403 Client Error: Forbidden for url: https://w...,1
2,403 Client Error: Forbidden for url: https://w...,1
3,403 Client Error: Forbidden for url: https://w...,1
4,403 Client Error: Forbidden for url: https://w...,1
5,403 Client Error: Forbidden for url: https://w...,1
6,403 Client Error: Forbidden for url: https://w...,1
7,403 Client Error: Forbidden for url: https://w...,1
8,403 Client Error: Forbidden for url: https://w...,1
9,403 Client Error: Forbidden for url: https://w...,1


In [20]:
con.close()

In [21]:
duckdb.close()

In [ ]:
# scrape player bios into database
def insert_bio_into_db(bio):
    con.execute("""
        INSERT INTO player_bios
        VALUES (?, ?, ?, ?)
    """, [bio["player_id"], bio["height_in"], bio["weight_lb"], bio["position"]])


already_bios = set(con.execute("SELECT DISTINCT player_id FROM player_bios").fetchall())

unique_players = con.execute("SELECT DISTINCT player_id FROM draft_classes").fetchall()

todo_bios = [pid for (pid,) in unique_players if (pid,) not in already_bios]

print(f"Need to scrape {len(todo_bios)} player bios")

CHECKPOINT_EVERY = 25

for i, pid in enumerate(todo_bios, 1):
    try:
        bio = scrape_player_bio(pid)
        insert_bio_into_db(bio)

        if i % CHECKPOINT_EVERY == 0:
            print(f"Scraped {i}/{len(todo_bios)} bios")

        time.sleep(random.uniform(8, 12))

    except Exception as e:
        print(f"Bio scrape failed for {pid}: {e}")


In [ ]:
# merge bios into rookie data
# bios = pd.read_csv("assets/player_bios.csv")


In [ ]:
# players missing minutes_total
# backfill from scratch
needs_backfill = rookie_caps_900[rookie_caps_900["minutes_total"].isna() | ~rookie_caps_900.columns.isin(["minutes_total"])]

backfill_results = []

for i, row in needs_backfill.iterrows():
    try:
        gamelog = load_rookie_gamelog(row.player_id, row.rookie_season)
        agg = aggregate_capped_minutes(gamelog, cap_minutes=900)

        backfill_results.append({
            "player_id": row.player_id,
            "rookie_season": row.rookie_season,
            "minutes_total": agg["minutes_total"],
            "hit_cap": agg["hit_cap"],
        })

        time.sleep(random.uniform(12,18))

    except Exception as e:
        print("Backfill failed:", row.player_id, row.rookie_season, e)


In [ ]:
# merge backfilled data
# do not overwrite main table until confirmed it works.

backfill_df = pd.DataFrame(backfill_results)

rookie_caps_900 = rookie_caps_900.merge(
    backfill_df,
    on=["player_id", "rookie_season"],
    how="left",
    suffixes=("", "_new")
)

rookie_caps_900["minutes_total"] = rookie_caps_900["minutes_total"].fillna(rookie_caps_900["minutes_total_new"])
rookie_caps_900["hit_cap"] = rookie_caps_900["hit_cap"].fillna(rookie_caps_900["hit_cap_new"])

rookie_caps_900 = rookie_caps_900.drop(columns=[c for c in rookie_caps_900.columns if c.endswith("_new")])
